<a href="https://colab.research.google.com/github/RedaGrace/ChatBot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from abc import ABCMeta, abstractmethod

import random
import json
import pickle
import numpy as np
import pandas as pd
import os

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

In [ ]:
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [3]:
lemmatizer = WordNetLemmatizer()

intents = pd.read_json('intents.json')

words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [5]:
documents[10:20]

[(['Good', 'Evening'], 'greeting'),
 (['hey'], 'greeting'),
 (['how', 'are'], 'greeting'),
 (['How', 'are', 'you', 'doing'], 'greeting'),
 (['Good', 'afternoon'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 ([], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye')]

In [6]:
words[0:10]

['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good']

In [7]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

In [8]:
for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        training.append([bag, output_row])

In [9]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [11]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
quit
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=20, batch_size=50, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
11097/11097 [==============================] - 33s 3ms/step - loss: 0.9930 - accuracy: 0.7398
Epoch 2/20
11097/11097 [==============================] - 34s 3ms/step - loss: 0.2487 - accuracy: 0.9180
Epoch 3/20
11097/11097 [==============================] - 32s 3ms/step - loss: 0.1903 - accuracy: 0.9357
Epoch 4/20
11097/11097 [==============================] - 34s 3ms/step - loss: 0.1611 - accuracy: 0.9451
Epoch 5/20
11097/11097 [==============================] - 34s 3ms/step - loss: 0.1430 - accuracy: 0.9510
Epoch 6/20
11097/11097 [==============================] - 33s 3ms/step - loss: 0.1309 - accuracy: 0.9546
Epoch 7/20
11097/11097 [==============================] - 34s 3ms/step - loss: 0.1222 - accuracy: 0.9572
Epoch 8/20
11097/11097 [==============================] - 33s 3ms/step - loss: 0.1136 - accuracy: 0.9597
Epoch 9/20
11097/11097 [==============================] - 34s 3ms/step - loss: 0.1079 - accuracy: 0.9616
Epoch 10/20
11097/11097 [==============================

In [12]:
model.save('chatbot_model.h5', hist)

In [13]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

In [14]:
def _clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [15]:
def _bag_of_words(sentence, words):
    sentence_words = _clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    return np.array(bag)


In [16]:
def _predict_class(sentence):
    p = _bag_of_words(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.1
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [17]:
def _get_response(ints, intents_json):
    try:
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag']  == tag:
                result = random.choice(i['responses'])
                break
    except IndexError:
        result = "I don't understand!"
    return result

In [18]:
def chat():
    print("Hi, How can i help you ?")
    while True:
        message = input("You: ")
        if message.lower() == 'quit':
            break
        ints = _predict_class(message)
        res = _get_response(ints, intents)
        print(res)

In [ ]:
chat()

Hi, How can i help you ?
You: hi
Good to see you again
You: how to register to the university?
You can join the university after fulfilling the conditions by evaluating the required papers in the affairs department and awaiting admission.
You: what if i missed an exam
The total of what you got from the pre-final exam marks is recorded for you, and in the event that you submit an excuse accepted by the Dean of the College within three days of the date of the exam, then you will record an incomplete mark, and you can apply for the final exam again, up to the end of the third week of the immediately following semester .
